# Pixel Drill for Indices outputs to CSV - Multi-site

| Authors:  | Bex Dunn|
|----------|----------------|
| Created: | March 6, 2019 |
| Last edited: | March 6, 2019 |

**Requirements:**

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles` 

`module load dea`

If you find an error or bug in this notebook, please either create an 'Issue' in the Github repository, or fix it yourself and create a 'Pull' request to contribute the updated notebook back into the repository (See the repository [README](https://github.com/GeoscienceAustralia/dea-notebooks/blob/master/README.rst) for instructions on creating a Pull request).

__Background:__ Data from the [Landsat](https://landsat.usgs.gov/about-landsat) 5,7 and 8 satellite missions are accessible through [Digital Earth Australia](http://www.ga.gov.au/about/projects/geographic/digital-earth-australia) (DEA).

__What does this notebook do?:__ This notebook takes a supplied CSV of site points. It runs a pixel drill through surface reflectance, calculates NDVI, Taselled cap wetness and greenness, and outputs a csv of values for each site and plots of each index for each site.

**Tags**: :index:`Landsat`,:index:`Landsat5`,:index:`Landsat7`,:index:`Landsat8`, :index:`pixeldrill`, :index:`DEAPlotting`, :index:`datacube.utils.geometry`, :index:`query`,:index:`Scripts`,:index:`tasseled_cap`, :index:`NDVI`,                                                                                                           :index:`DEADataHandling`, :index:`DEAPlotting`, :index:`load_clearlandsat`

import some modules

In [4]:
import datacube
import datetime
import fiona
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio.mask
import rasterio.features
import shapely
import seaborn as sns
import sys
import xarray as xr

import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from datacube.storage import masking
from datacube.utils import geometry

sys.path.append('../10_Scripts')
import DEADataHandling, DEAPlotting, TasseledCapTools, BandIndices

dc = datacube.Datacube(app='pixel drill')

%load_ext autoreload

%autoreload 2

#set up file to open 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
inpath = '/g/data/r78/rjd547/Melbourne_water/150 points to check.csv'

### Convert csv latitude and longitude values into a geopandas geodatafrome

sites = pd.read_csv(inpath, delimiter=",")
#turn csv geometry into geopandas dataframe geometry using lambda functions and shapely
sites['geometry']=sites.apply(lambda z: shapely.geometry.Point(z.y, z.x), axis=1)
sites = gpd.GeoDataFrame(sites)
sites.head()

#define an output location
output_loc = '/g/data/r78/rjd547/Melbourne_water/points/point_'

### run this for multiple sites

for site in range(0,(len(sites)),1):    
    print(f'running for index {site}')
    lon = sites['x'][site]
    lat = sites['y'][site]
    query = {'lat':lat, 
          'lon':lon}               
    #print(query)
    outfilename=f'{output_loc}{lat}_{lon}.csv'
    print (outfilename)

    try:
        ls578 = DEADataHandling.load_clearlandsat(dc=dc, query=query, product='nbart', ls7_slc_off=True)
    except:
        print ('nah')
        ### Calculate NDVI 

    ### Calculate the tasseled cap indices
    tci = TasseledCapTools.thresholded_tasseled_cap(ls578,wetness_threshold=-350, drop=True , drop_tc_bands=False)
    tcw = tci.wetness

     ### create and fill pandas dataframe to write to csv

    #drop dimensions of length 1
    tcw = tcw.squeeze('x')
    tcw = tcw.squeeze('y')

    #     #make a new dataframe using the data from the xarray of ndvi
    INDICIES = pd.DataFrame(data=tcw.data, index=tcw.time.values,columns=['TCW'])
    INDICIES.to_csv(outfilename)

!ls /g/data/r78/rjd547/Melbourne_water/points/*.csv    

running for index 0
/g/data/r78/rjd547/Melbourne_water/points/point_-37.72228993_144.20836609999998.csv
Loading ls5
    Loading 295 filtered ls5 timesteps
Loading ls7
    Loading 299 filtered ls7 timesteps
Loading ls8
    Loading 134 filtered ls8 timesteps
Combining and sorting ls5, ls7, ls8 data
    Replacing invalid -999 values with NaN (data will be coerced to float64)
running for index 1
/g/data/r78/rjd547/Melbourne_water/points/point_-37.75835133_144.4445868.csv
Loading ls5
    Loading 295 filtered ls5 timesteps
Loading ls7
    Loading 299 filtered ls7 timesteps
Loading ls8
    Loading 135 filtered ls8 timesteps
Combining and sorting ls5, ls7, ls8 data
    Replacing invalid -999 values with NaN (data will be coerced to float64)
running for index 2
/g/data/r78/rjd547/Melbourne_water/points/point_-37.55447281_144.60000480000002.csv
Loading ls5
    Loading 295 filtered ls5 timesteps
Loading ls7
    Loading 301 filtered ls7 timesteps
Loading ls8
    Loading 135 filtered ls8 timesteps
